In [1]:
pip install -qU "langchain[google-genai]"

Note: you may need to restart the kernel to use updated packages.


  You can safely remove it manually.

[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import os
import getpass

In [3]:
if not os.environ.get("GOOGLE_API_KEY"):
    os.environ["GOOGLE_API_KEY"]=getpass.getpass("Enter the API key for google gemini:")
    

In [5]:
from langchain.chat_models import init_chat_model
model=init_chat_model("gemini-2.5-flash",model_provider="google_genai")

In [6]:
response=model.invoke("What is Langchain")
print(response.content)

LangChain is an open-source **framework** designed to help developers build applications powered by Large Language Models (LLMs). It provides a structured way to combine LLMs with other components to create more complex and capable applications than a standalone LLM could achieve.

Think of it this way: LLMs are incredibly powerful "brains," but they often need help with memory, accessing external knowledge, or performing specific actions. LangChain acts as the **orchestration layer** that connects the LLM brain to all these necessary "limbs" and "senses."

### Key Problems LangChain Solves:

1.  **Limited Context Window:** LLMs can only remember a certain amount of conversation. LangChain helps integrate "memory" to maintain long-term context.
2.  **Lack of External Knowledge:** LLMs are trained on a fixed dataset. LangChain enables them to access up-to-date information from the internet, databases, or custom documents.
3.  **Inability to Perform Actions:** LLMs can generate text, but

#### Creating basic agent

In [7]:
from langchain.agents import create_agent

In [10]:
def get_weather(city:str)->str:
    """ Get weather for a given city."""

    return f"It is cloudy in {city}"


In [11]:
agent=create_agent(
    model=model,
    tools=[get_weather],
    system_prompt="You are a helpful assistant",
)

In [15]:
response1=agent.invoke(
    {"messages":[{"role":"user","content":"What is weather in Nepal"}]}
)

In [16]:
print(response1)

{'messages': [HumanMessage(content='What is weather in Nepal', additional_kwargs={}, response_metadata={}, id='5ffe6958-2998-4753-bb92-92ddbf1c7c99'), AIMessage(content=[{'type': 'text', 'text': 'Nepal is a country. I need a specific city to get the weather. Can you please tell me a city in Nepal?', 'extras': {'signature': 'CpcFAdHtim/WUg8XwPdRYd5H4VgKLR+1E4qNOVu5pc3gm0CZT/ZuyHX2/hR0g9Ogq2QXEIE+jrCT5ceDZYGRS0x/R8VT2oN9BS7htcUWiMC/LM7CnUI0TCj5cSMg3JE96FR31EeS46sOBxDBP2cjWKVwhaU0e8CSGPWj7O0yU6uWmmc/nZJfGvPjsdfjtV24QTwtdA37TE0/KC5VZgNKovg93h5CpTGlzHV7+yK5AWnARbbjtXpgrLlCG5A7bgEvrlAkl6aRCk0Y+85Z8T+HJ5JX18yf35i++xc3jA/UNb1hZhbOX9hul5cy6JkeUMJW3gv/E6bU/XwbZGmzC3jN4vNEc11VW5xdd6Dv2SsKfnldiLQQJ2zNyNaB/taDaktkeRWlpSuC712CSzoXQXpka+tb281pzMopj58vi9g8G1gFDV/FvzXU3smUREVcETjW3AHBK41Y8XvtMFOmouzIcbYwDvPzlB7S4k3Rp35EOctethWJ02D+S4WM2g1uNHBpwF22AQs0bdkVCvWLuqBbD6N2fRqSfwT6IHVVf85cDlCWt64evYqZUYT6I3yI3FT2GxsEJMrv/HUoEcYRC/VmKVP/ZDN1YghqiTrkKbEL42Y7CEWyIBkrJoFB6GE+x3/Ub9OC3zb1mYnozdVkVaev0i9Z4vuIF/4J4U

#### Using Language Models

In [18]:
from langchain_core.messages import HumanMessage,SystemMessage

message=[
    SystemMessage(content="Translate the sentence from English to Japnese"),
    HumanMessage(content="Good morning")
]
res=model.invoke(message)
print(res)

content="**おはようございます** (Ohayō gozaimasu)\n\n*   **おはよう** (Ohayō) is also used, but it's more casual, typically among family, close friends, or colleagues in the same team/department.\n*   **おはようございます** (Ohayō gozaimasu) is the polite and standard form, appropriate for most situations." additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'grounding_metadata': {}, 'model_provider': 'google_genai'} id='lc_run--37ea3965-62f6-4012-839f-64f89455277d-0' usage_metadata={'input_tokens': 11, 'output_tokens': 151, 'total_tokens': 162, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 72}}


#### Prompt Template
 take in raw user input and return data (a prompt) that is ready to pass into a language model.

In [19]:
from langchain_core.prompts import ChatPromptTemplate

system_template="Translate te following from English to {language}"
prompt_template=ChatPromptTemplate.from_messages([("system",system_template),("user","{text}")])

In [22]:
prompt=prompt_template.invoke({"language":"Korean","text":"hi"})
prompt

ChatPromptValue(messages=[SystemMessage(content='Translate te following from English to Korean', additional_kwargs={}, response_metadata={}), HumanMessage(content='hi', additional_kwargs={}, response_metadata={})])

In [23]:
res=model.invoke(prompt)
print(res)


content='안녕하세요 (annyeonghaseyo)' additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'grounding_metadata': {}, 'model_provider': 'google_genai'} id='lc_run--d27b97d5-5563-49df-bdbd-676a5581a822-0' usage_metadata={'input_tokens': 9, 'output_tokens': 43, 'total_tokens': 52, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 36}}


## Semantic Seach Engine

#### Document Loader

In [24]:
pip install langchain-community pypdf

   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ---- ----------------------------------- 0.3/2.5 MB ? eta -:--:--
   -------- ------------------------------- 0.5/2.5 MB 840.2 kB/s eta 0:00:03
   -------- ------------------------------- 0.5/2.5 MB 840.2 kB/s eta 0:00:03
   ------------ --------------------------- 0.8/2.5 MB 932.9 kB/s eta 0:00:02
   ---------------- ----------------------- 1.0/2.5 MB 988.0 kB/s eta 0:00:02
   -------------------- ------------------- 1.3/2.5 MB 959.4 kB/s eta 0:00:02
   ------------------------ --------------- 1.6/2.5 MB 999.0 kB/s eta 0:00:01
   ---------------------------- ----------- 1.8/2.5 MB 1.0 MB/s eta 0:00:01
   ---------------------------- ----------- 1.8/2.5 MB 1.0 MB/s eta 0:00:01
   --------------------------------- ------ 2.1/2.5 MB 978.7 kB/s eta 0:00:01
   -----------------------


[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [27]:
from langchain_community.document_loaders import PyPDFLoader
file_path="basic-text.pdf"
loader=PyPDFLoader(file_path)
docs=loader.load()
print(len(docs))


1


In [29]:
print(f"{docs[0].page_content[:200]}")
print(f"{docs[0].metadata}")

Sample Document for PDF Testing
Introduction
This is a simple document created to test basic PDF functionality. It includes various text formatting
options to ensure proper rendering in PDF readers.
T
{'producer': 'Skia/PDF m126', 'creator': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36', 'creationdate': '2024-07-09T13:31:41+00:00', 'title': 'Sample Document for PDF Testing', 'moddate': '2024-07-09T13:31:41+00:00', 'source': 'basic-text.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1'}


#### Splitter

In [30]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
splitter=RecursiveCharacterTextSplitter(chunk_size=100,chunk_overlap=10,add_start_index=True)
all_splits=splitter.split_documents(docs)
len(all_splits)

12

In [32]:
all_splits[6]

Document(metadata={'producer': 'Skia/PDF m126', 'creator': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36', 'creationdate': '2024-07-09T13:31:41+00:00', 'title': 'Sample Document for PDF Testing', 'moddate': '2024-07-09T13:31:41+00:00', 'source': 'basic-text.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1', 'start_index': 410}, page_content="Item 1\nItem 2\nItem 3\nAnd here's an ordered list:\n1. First item\n2. Second item\n3. Third item\nQuote")

#### Embeddings

In [36]:
pip install -qU langchain-google-genai

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [37]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
embeddings=GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001")

In [38]:
vector_1=embeddings.embed_query(all_splits[0].page_content)
vector_2=embeddings.embed_query(all_splits[1].page_content)

print(len(vector_1))

3072


In [39]:
print(vector_1[:10])

[-0.009608505293726921, -0.005367163568735123, 0.015974808484315872, -0.07413351535797119, -0.0062061939388513565, 0.012328092940151691, 0.011254881508648396, -0.016206800937652588, 0.0011252409312874079, 0.01037000585347414]


#### Vector Storage

In [40]:
pip install -qU langchain-qdrant

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [41]:
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from qdrant_client.models import Distance,VectorParams

In [44]:
client=QdrantClient("http://localhost:6333")


In [45]:
vector_size=len(embeddings.embed_query("sample text"))
if not client.collection_exists("test"):
    client.create_collection(
        collection_name="test",
        vectors_config=VectorParams(size=vector_size,distance=Distance.COSINE)
    )
vector_store=QdrantVectorStore(
        client=client,
        collection_name="test",
        embedding=embeddings,
    )

In [46]:
ids=vector_store.add_documents(documents=all_splits)

In [56]:
res=vector_store.similarity_search("What is this pdf about?")
print(res[0])

page_content='This sample PDF file is provided by Sample-Files.com. Visit us for more sample files and resources.' metadata={'producer': 'Skia/PDF m126', 'creator': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36', 'creationdate': '2024-07-09T13:31:41+00:00', 'title': 'Sample Document for PDF Testing', 'moddate': '2024-07-09T13:31:41+00:00', 'source': 'basic-text.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1', 'start_index': 816, '_id': '28175c60-a67f-472e-97aa-8e0366c34596', '_collection_name': 'test'}


In [58]:
res = await vector_store.asimilarity_search("What is this pdf about?")

print(res[0])

page_content='This sample PDF file is provided by Sample-Files.com. Visit us for more sample files and resources.' metadata={'producer': 'Skia/PDF m126', 'creator': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36', 'creationdate': '2024-07-09T13:31:41+00:00', 'title': 'Sample Document for PDF Testing', 'moddate': '2024-07-09T13:31:41+00:00', 'source': 'basic-text.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1', 'start_index': 816, '_id': '28175c60-a67f-472e-97aa-8e0366c34596', '_collection_name': 'test'}


### Building weather agent